In [1]:
!pip install torchmetrics
!pip install lightning
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 10.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [2]:
# For loading dataset
import pandas as pd
import numpy as np
from sklearn.utils import resample,shuffle
from sklearn.model_selection import train_test_split

import os
import re
import ast
import torch

import torchmetrics
from torch import nn
import pytorch_lightning as pl
import torch.nn.functional as F
import torchvision.models as models
from transformers.optimization import AdamW
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from transformers import get_linear_schedule_with_warmup
from transformers import XLMRobertaConfig, XLMRobertaTokenizer, XLMRobertaModel

# Loss function
from torchvision.ops import sigmoid_focal_loss

# logger
import wandb
from lightning.pytorch.loggers import WandbLogger

# plotting
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# visual
from tqdm import tqdm
import gc

from transformers import AutoTokenizer, AutoModelForMaskedLM
from torchmetrics.regression import MeanSquaredError
from collections import Counter

# #nltk
# from sklearn.feature_extraction.text import CountVectorizer
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

# train & val
data = pd.read_csv("/content/drive/MyDrive/ytcomment/trainValfortrain.csv", usecols= ['firstTopComment', 'soft_prob', 'one_hot_encoding'])

# test
test = pd.read_csv("/content/drive/MyDrive/ytcomment/SemEval2017-task4-test.subtask-A.english.txt", delimiter = "\t", header= None, names= ['id', 'soft_label', 'firstTopComment'], usecols= ['firstTopComment', 'soft_label'])

# youtube test
pred_yt = pd.read_csv("/content/drive/MyDrive/ytcomment/testfortrain.csv", usecols= ['firstTopComment', 'soft_label', 'one_hot_encoding'])

# emoji test
pred_emoji = pd.read_csv("/content/drive/MyDrive/ytcomment/testemoji.csv", header= 0, names= ['index', 'firstTopComment', 'text', 'Tensor', 'soft_label'], usecols= ['firstTopComment', 'soft_label'])

In [5]:
def column_processing(dataset, soft_prob_p=False, one_hot_encoding_p=False, class_int_w=False, class_int_wo=False):
    # Convert soft_prob to int
    if soft_prob_p:
        dataset['soft_prob'] = dataset['soft_prob'].apply(ast.literal_eval)

    # Convert one_hot_encoding to int
    if one_hot_encoding_p:
        dataset['one_hot_encoding'] = dataset['one_hot_encoding'].astype(str).apply(ast.literal_eval)

    # Use soft_label to convert one hot encoding
    if class_int_w:
        label_to_one_hot = {'positive': [1, 0, 0], 'neutral': [0, 1, 0], 'negative': [0, 0, 1]}
        dataset['one_hot_encoding'] = dataset['soft_label'].map(label_to_one_hot)

    # Use soft_label as its class_int
    elif class_int_wo:
        dataset['class_int'] = dataset['soft_label']

column_processing(data, soft_prob_p=True, one_hot_encoding_p=True)
column_processing(pred_yt, one_hot_encoding_p=True)
column_processing(test, class_int_w=True)
column_processing(pred_emoji, class_int_w=True)

In [6]:
# prevent data shapes in special distribution
data = shuffle(data)

train, val = train_test_split(data, train_size= 0.9, shuffle= True, random_state=42)
print(f"Train: {len(train)}, Validation: {len(val)}, test: {len(test)}")

Train: 18000, Validation: 2000, test: 11906


In [7]:
# DataModule Parameters
access_token = ''  # Access token for tokenizer & model
batch_size = 8
max_token_len = 100  # Maximum length of tokenized sequences

# Model Training Parameter
lr = 2e-5
dropout = 0.5
n_classes = 3
optimizer = AdamW
steps_per_epoch=len(train) // batch_size
total_training_steps = steps_per_epoch * 2
warmup_steps = total_training_steps // 5
print(f'steps_per_epoch: {steps_per_epoch}, total_training_steps: {total_training_steps}, warmup_steps: {warmup_steps}')

steps_per_epoch: 2250, total_training_steps: 4500, warmup_steps: 900


In [8]:
class EncodeDataset(Dataset):
    def __init__(self, data: pd.DataFrame, label_column: str, access_token: str, max_token_len:int):
        self.tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base", token=access_token)
        self.data = data
        self.label_column = label_column
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.data)

    def preprocess(self, text):
        patterns = [
            (r'http\S+', '', 0),  # remove links
            (r'#', '', 0),        # remove hashtags
            (r'@\S+', '', 0),     # remove mentions
            (r'\ufe0f', '', re.U) # remove unicode character
        ]
        for pattern, repl, flg in patterns:
            text = re.sub(pattern, repl, text, flg)
        return text

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]
        comment_text = self.preprocess(data_row.firstTopComment)
        labels = data_row[self.label_column]

        encoding = self.tokenizer(
            text=comment_text,
            add_special_tokens=True,
            max_length=self.max_token_len,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_token_type_ids=False,  # XLM-RoBERTa doesn't use token type IDs
            return_tensors='pt',
        )

        return dict(
            input_ids=encoding['input_ids'].squeeze(),
            attention_mask=encoding['attention_mask'].squeeze(),
            labels=torch.FloatTensor(labels)  # Ensure labels is a tensor
        )

In [9]:
class DataModule(pl.LightningDataModule):
    def __init__(self, train_df: pd.DataFrame, val_df: pd.DataFrame, test_df: pd.DataFrame, pred_df: pd.DataFrame, batch_size: int, max_token_len: int, access_token: str):
        super().__init__()
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        self.pred_df = pred_df
        self.batch_size = batch_size
        self.access_token = access_token
        self.max_token_len = max_token_len

    def setup(self, stage=None):
        if stage == "fit" or stage is None:
            self.train_dataset = EncodeDataset(self.train_df, label_column='soft_prob', access_token=self.access_token, max_token_len=self.max_token_len)
            self.val_dataset = EncodeDataset(self.val_df, label_column='soft_prob', access_token=self.access_token, max_token_len=self.max_token_len)
        if stage == "test" or stage is None:
            self.test_dataset = EncodeDataset(self.test_df, label_column='one_hot_encoding', access_token=self.access_token, max_token_len=self.max_token_len)
        if stage == "predict" or stage is None:
            self.pred_dataset = EncodeDataset(self.pred_df, label_column='one_hot_encoding', access_token=self.access_token, max_token_len=self.max_token_len)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=4)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=4)

    def predict_dataloader(self):
        return DataLoader(self.pred_dataset, batch_size=self.batch_size, num_workers=4)

data_module = DataModule(train, val, test, pred_yt, batch_size=batch_size, max_token_len=max_token_len, access_token=access_token)


In [10]:
class CommentClassifier(pl.LightningModule):
    # def __init__(self, n_classes: int, n_training_steps = None, n_warm_steps = None):
    def __init__(self, lr, optimizer, dropout, n_classes: int, access_token: str, n_training_steps = None, n_warm_steps = None):
        super().__init__()

        # xlm-RoBERTa
        self.access_token = access_token
        self.dropout = dropout
        configuration = XLMRobertaConfig(hidden_dropout_prob = self.dropout, add_pooling_layer = True)

        # It's very important to return_dict here!
        # return_dict (bool, optional) — Whether or not to return a ModelOutput instead of a plain tuple.
        #https://github.com/huggingface/transformers/tree/main/src/transformers/models/xlm_roberta
        self.xlmroberta = XLMRobertaModel.from_pretrained("xlm-roberta-base", configuration, return_dict = True , token=self.access_token)
        self.hidden_size = configuration.hidden_size

        #
        self.fc1 = nn.Linear(self.hidden_size, 3)
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warm_steps

        # CrossEntropyLoss default reduction is "mean"
        # ref: https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
        self.criterion1 = nn.CrossEntropyLoss()
        self.criterion2 = nn.CrossEntropyLoss(reduction = 'none')

        # temperature for softmax
        # https://arxiv.org/pdf/1503.02531
        self.temp = 1

        self.lr = lr
        self.optimizer = optimizer
        self.droplayers = nn.Dropout(p=self.dropout)


        self.train_metrics = nn.ModuleDict({'acc': torchmetrics.Accuracy(task="multiclass", num_classes=3),
                                            'f1': torchmetrics.F1Score(task="multiclass", num_classes=3, average=None),
                                            'auroc': torchmetrics.AUROC(task="multiclass", num_classes=3),
                                            'recall': torchmetrics.Recall(task="multiclass", average='macro', num_classes=3),
                                            'precision': torchmetrics.Precision(task="multiclass", average='macro', num_classes=3)})
        self.valid_metrics = nn.ModuleDict({'acc': torchmetrics.Accuracy(task="multiclass", num_classes=3),
                                            'f1': torchmetrics.F1Score(task="multiclass", num_classes=3, average=None),
                                            'auroc': torchmetrics.AUROC(task="multiclass", num_classes=3),
                                            'recall':torchmetrics.Recall(task="multiclass", average='macro', num_classes=3),
                                            'precision': torchmetrics.Precision(task="multiclass", average='macro', num_classes=3)})
        self.test_metrics = nn.ModuleDict({'acc': torchmetrics.Accuracy(task="multiclass", num_classes=3),
                                           'recall':torchmetrics.Recall(task="multiclass", average='macro', num_classes=3),
                                           'precision': torchmetrics.Precision(task="multiclass", average='macro', num_classes=3),
                                           'f1': torchmetrics.F1Score(task="multiclass", num_classes=3, average=None)})

        # save hyper-parameters to self.hparamsm auto-logged by wandb
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels = None):
        output = self.xlmroberta(input_ids, attention_mask,return_dict=True)
        output = self.fc1(output.pooler_output)
        output = nn.functional.softmax(output / self.temp, dim=-1)
        return output

    def argmaxDist(self, output, label):
        # Predict: Take the max from pred distribution
        argmax_result = torch.argmax(output, dim=1)

        # Taget: Take the max label from soft_prob (dist from gpt)
        argmax_label = torch.argmax(label, dim = 1)
        return argmax_result, argmax_label

    def loss_and_log(self, output, label, flg):
        if label is None:
            return None

        # Map flg to loss function and log key
        loss_funcs = {
            'train': ('train_loss', lambda o, l: sigmoid_focal_loss(o, l, gamma=7, reduction='mean')),
            'val': ('val_loss', lambda o, l: sigmoid_focal_loss(o, l, gamma=7, reduction='mean')),
            'test': ('test_loss', self.criterion1)
        }

        if flg in loss_funcs:
            log_key, loss_func = loss_funcs[flg]
            loss = loss_func(output, label)
            self.log(log_key, loss, prog_bar=True, on_epoch=True)
            return loss

        return None

    def eval_metrics(self, output, argmax_result, argmax_label, flg):
        argmax_result, argmax_label, output = map(lambda x: x.detach(), (argmax_result, argmax_label, output))

        metrics = self.train_metrics if flg == "train" else self.valid_metrics if flg == "val" else self.test_metrics if flg == "test"

        # AvgRec
        rec = metrics['recall'](argmax_result, argmax_label)
        self.log(f'{flg}_AvgRec', rec, prog_bar=True, on_epoch=True)

        # Accuracy and Error
        acc = metrics['acc'](argmax_result, argmax_label)
        self.log(f'{flg}_acc', acc, prog_bar=True, on_epoch=True)
        self.log(f'{flg}_error', 1 - acc, prog_bar=True, on_epoch=True)

        # Precision
        pre = metrics['precision'](argmax_result, argmax_label)
        self.log(f'{flg}_precision', pre, prog_bar=True, on_epoch=True)

        # AUROC (Skip for 'test' flag)
        if flg != 'test':
            metrics['auroc'].update(output, argmax_label)
            self.log(f'{flg}_auroc', metrics['auroc'], prog_bar=True, on_epoch=True)

        # F1: Only caculate the average of "Positive" and "Negative"
        f1 = metrics['f1'](argmax_result, argmax_label)
        f1_positive, f1_negative = f1[0].item(), f1[2].item()
        Fpn = 0.5 * (f1_positive + f1_negative)
        self.log(f'{flg}_f1', Fpn, prog_bar=True, on_epoch=True)

    def write2file(self, output, argmax_result, filename):
        output_forwrite = output.cpu().numpy()

        # Distribution
        with open(f'{filename}_dist.txt', "a") as file:
            for pred in output_forwrite:
                line = ', '.join(map(str, pred))
                file.write(line + "\n")

        # Label
        with open(f'{filename}_label.txt', "a") as file:
            for pred in argmax_result:
                file.write(f"{pred.item()}\n")

    def step(self, batch, batch_idx, flg):
        input_id = batch['input_ids']
        attention_mask = batch['attention_mask']
        label = batch['labels']
        output = self(input_id, attention_mask, label)

        if flg == "train" or flg == "val":
          # Get "Positive", "Neutral" or "Negative" from prediction and target
          argmax_result, argmax_label = self.argmaxDist(output, label)

          # caculate and record loss
          loss = self.loss_and_log(output, label, flg)

          # evaluation
          self.eval_metrics(output, argmax_result, argmax_label, flg)

          if flg =='test':
            # write the pred distribution and pred label to file
            self.write2file(output, argmax_result, "test_semeval")
        else:
            # Doesn't need to evaluate and caculate loss, so just take the argmax_result
            argmax_result = torch.argmax(output, dim=1)

            # write the pred distribution and pred label to file
            self.write2file(output, argmax_result, "pred_yt")
            return None

        return loss



    def training_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, "train")

    def validation_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, "val")

    def test_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, "test")

    def predict_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, "pred")

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr = self.lr)

        scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=self.n_warmup_steps,
        num_training_steps=self.n_training_steps
        )
        return dict(
        optimizer=optimizer,
        lr_scheduler=dict(
            scheduler=scheduler,
            interval='step'
        )
        )


In [11]:
model = CommentClassifier(n_classes = n_classes, lr = lr, dropout = dropout, optimizer = optimizer, n_training_steps = total_training_steps, n_warm_steps = warmup_steps, access_token = access_token)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [12]:
import gc

wandb.init(project="sentiment-project", name = "test_final")


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [13]:
checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min")
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

logger = WandbLogger()
trainer = pl.Trainer(
    logger=logger,
    callbacks=[early_stopping_callback, checkpoint_callback],
    max_epochs=5,
    # accelerator="cpu",
    log_every_n_steps = 1
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [14]:
trainer.fit(model, datamodule = data_module)

gc.collect()

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | xlmroberta    | XLMRobertaModel  | 278 M  | eval 
1 | fc1           | Linear           | 2.3 K  | train
2 | criterion1    | CrossEntropyLoss | 0      | train
3 | criterion2    | CrossEntropyLoss | 0      | train
4 | droplayers    | Dropout          | 0      | train
5 | train_metrics | ModuleDict       | 0      | train
6 | valid_metrics | ModuleDict       | 0      | train
7 | test_metrics  | ModuleDict       | 0      | train
--------------

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

150

In [15]:
# trainer.test(datamodule = data_module,ckpt_path='/content/drive/MyDrive/epoch=4-step=11250.ckpt')
trainer.test(datamodule = data_module,ckpt_path='best')

INFO: Restoring states from the checkpoint path at ./lightning_logs/d5s6kn50/checkpoints/epoch=1-step=4500.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at ./lightning_logs/d5s6kn50/checkpoints/epoch=1-step=4500.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at ./lightning_logs/d5s6kn50/checkpoints/epoch=1-step=4500.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at ./lightning_logs/d5s6kn50/checkpoints/epoch=1-step=4500.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

[{}]

In [16]:
# trainer.predict(datamodule = data_module,ckpt_path='/content/drive/MyDrive/epoch=4-step=11250.ckpt')
trainer.predict(datamodule = data_module,ckpt_path='best')

INFO: Restoring states from the checkpoint path at ./lightning_logs/d5s6kn50/checkpoints/epoch=1-step=4500.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at ./lightning_logs/d5s6kn50/checkpoints/epoch=1-step=4500.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at ./lightning_logs/d5s6kn50/checkpoints/epoch=1-step=4500.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at ./lightning_logs/d5s6kn50/checkpoints/epoch=1-step=4500.ckpt


Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/prediction_loop.py:255: predict returned None if it was on purpose, ignore this warning...


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [17]:
wandb.finish()

epoch,▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▃▆▆▆▆▆▆▆▆▆▆██████████
train_AvgRec_epoch,▁▆██
train_AvgRec_step,▁▁▅▃▅▃▄██▄▆▇█▅█▂▆▃▃▇▃█▆▅▅▅█▇▇▇▆▇▂▄▇█▆█▄▂
train_acc_epoch,▁▆██
train_acc_step,▅▁▅▆▃▃▃██▅▆▆█▃█▆▅▃▅▆▃█▆▅▆▆█▆▅▆▆▆▆▅▆█▆█▆▃
train_auroc_epoch,▁▆██
train_error_epoch,█▃▁▁
train_error_step,▅█▅▃▆▆▆▁▁▅▃▃▁▆▁▃▅▆▅▃▆▁▃▅▃▃▁▃▅▃▃▃▃▅▃▁▃▁▃▆
train_f1_epoch,▁▆██
train_f1_step,▂▁▁▂▁▂▅█▃▅▆▇▃▁▃▂▂▅▆▇▂█▂▂█▂██▇▇▆▂▂▇▇▃▆▃▇▂
train_loss_epoch,█▄▁▁


### Sweep

In [ ]:
# if __name__ == "__main__":
#     sweep_config = {
#         "method": "random",
#         "metric": {"goal": "minimize", "name": "val_loss"},
#         "parameters": {
#             "learning_rate": {"min": 1e-5, "max": 1e-3},
#             "dropout": {'values': [0.4, 0.5, 0.6, 0.7, 0.8]},
#             'optimizer': {'values': ['AdamW', 'adam']},
#     },
# }
#     sweep_id = wandb.sweep(sweep_config, project="sentiment-project")
#     wandb.agent(sweep_id, function=train, count=10)